In [2]:
import datasets

In [3]:
#carrega natural questions

from datasets import load_dataset
dataset = load_dataset('json', data_files="simplified-nq-train.jsonl")

C:\Users\dudua\anaconda3\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [11]:
#carrega modelo de tradução

from transformers import pipeline
pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt")


C:\Users\dudua\anaconda3\lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'O Tom tentou esfaquear-me.'}]


In [59]:
df = pd.DataFrame(columns = ['title', 'context','question', 'answer'])

In [129]:
sample = dataset['train'][0]

In [130]:
titulo = sample['document_text']

In [131]:
re.findall(r'(?<=<h1>)(.+?)(?=</h1>)', titulo, re.IGNORECASE)[0]

' Email marketing '

In [132]:
sample['annotations']

[{'yes_no_answer': 'NONE',
  'long_answer': {'start_token': 1952,
   'candidate_index': 54,
   'end_token': 2019},
  'short_answers': [{'start_token': 1960, 'end_token': 1969}],
  'annotation_id': 5.931654502200276e+17}]

In [133]:
doc = sample['document_text']

In [134]:
sample['annotations'][0]['long_answer']

{'start_token': 1952, 'candidate_index': 54, 'end_token': 2019}

In [135]:
init = sample['annotations'][0]['short_answers'][0]['start_token']
end = sample['annotations'][0]['short_answers'][0]['end_token']

In [136]:
" ".join(doc.split(" ")[init:end])

"a newsletter sent to an advertising firm 's customers"

In [29]:
#sample['question_text']

'what happened to the lost settlement of roanoke'

In [39]:
import pandas as pd
import re
import sys
import time

In [127]:
def translate(size=10, dataset=dataset):
    df = pd.DataFrame(columns = ['title', 'question', 'long_answer', 'short_answer'])
    small_set = dataset['train']
    for i in range(size):

        doc = re.sub(r'''<(?:"[^"]*"['"]*|'[^']*'['"]*|[^'">])+>''','',small_set[i]['document_text'])
        init_long = small_set[i]['annotations'][0]['long_answer']['start_token']
        end_long = small_set[i]['annotations'][0]['long_answer']['end_token']
        if len(small_set[i]['annotations'][0]['short_answers']) != 0:
            init_short = small_set[i]['annotations'][0]['short_answers'][0]['start_token']
            end_short = small_set[i]['annotations'][0]['short_answers'][0]['end_token']
        else:
            init_short = 0
            end_short  = 0
        question = small_set[i]['question_text']
        long_answer = " ".join(doc.split(" ")[init_long:end_long])
        short_answer = " ".join(doc.split(" ")[init_short:end_short])
        title = re.findall(r'(?<=<h1>)(.+?)(?=</h1>)', small_set[i]['document_text'], re.IGNORECASE)[0]
        if len(long_answer) > 0:  
            title = pipe('>>por<< ' + title)[0]['translation_text']
            long_answer= pipe('>>por<< ' + long_answer)[0]['translation_text']
            if len(short_answer) != 0:
                short_answer= pipe('>>por<< ' + short_answer)[0]['translation_text']
            question= pipe('>>por<< ' + question)[0]['translation_text']

            
            
        else:
            pass
            
        new_row = {'title': [title], 'question': [question], 'long_answer': [long_answer], 'short_answer': [short_answer]}
        df_aux = pd.DataFrame(new_row)
        df = pd.concat([df, df_aux])

        #configuração do terminal
        sys.stdout.write(f'\rProgress : {((i+1)/size)*100}%')
        sys.stdout.flush()
        
    df = df.loc[df['long_answer'] != '']
    df.reset_index(drop=True, inplace=True)
    return df
    




In [139]:
%%time
traduzido = translate(size=20)

Progress : 100.0%00000000001%Wall time: 1min 51s


In [154]:
squad = {
    "version": "0.1",
    "data": [
        
    ],
}

In [159]:
import json
for passage in json.loads(traduzido.to_json(orient='records')):
    estrutura = {
        "title": passage['title'],
        "paragraphs": [
            {
            "context": passage['long_answer'],
            "qas": [
                {
                    "id":0,
                    "question": passage['question'],
                    "answers": [
                        {
                        'text':passage['short_answer'],
                        }
                        
                    ],
                },
            ],
            },
        ],
    }
    squad['data'].append(estrutura)
    

In [163]:
json_data =squad
file_path = "data.json"
with open(file_path, "w") as json_file:
    json.dump(json_data, json_file, indent=4)

print(f"Variable saved as JSON")

Variable saved as JSON
